In [ ]:
from hardware.opentrons.http_communications import OpentronsAPI
from hardware.opentrons.configuration import Configuration

api = OpentronsAPI()
api.initialise()
config = Configuration(http_api=api)
labware = config.load_labware()
pipettes = config.load_pipettes()
containers = config.load_containers()
right_pipette = pipettes["right"]

AttributeError: 'Configuration' object has no attribute 'logger'

In [ ]:
def dispense(self, volume: float, source: "Container", log = True):
        if (self.volume_mL * 1e3) + volume > self.MAX_VOLUME:
            self.protocol_logger.warning("Overflowing of container!")
            return
        self.volume_mL += volume * 1e-3
        self.update_liquid_height(volume_mL=self.volume_mL)

        # case 1: container is empty
        if self.solution_name == "empty":
            self.solution_name = source.solution_name
            self.concentration = source.concentration

        # case 2: container contains water and solution is added from source
        elif self.solution_name == "water" and source.solution_name != "water":
            self.solution_name = source.solution_name
            self.concentration = (float(source.concentration) * volume*1e-3) / self.volume_mL
            self.dilution_factor = source.concentration / self.concentration

        # case 3: containers contains solution and water is added from source
        elif self.solution_name != "water" and source.solution_name == "water":
            self.concentration = (
                float(self.concentration) * volume * 1e-3
            ) / self.volume_mL
            self.dilution_factor = source.concentration / self.concentration

        # case 4: container contains solution and same solution, but different concentration, is added from source
        elif self.solution_name == source.solution_name:
            if self.solution_name != "water":
                n_source_mM = float(source.concentration) * volume * 1e-3
                n_container_mM = float(self.concentration) * (self.volume_mL - volume * 1e-3)
                self.concentration = (
                    n_source_mM + n_container_mM
                ) / self.volume_mL
            else:
                pass

        # case 5: other cases #TODO extend to mixtures
        else:
            if log:
                self.container_logger.warning(f"The case of adding {source.solution_name} of {source.concentration} mM from source, to a container with {self.solution_name} of {self.concentration} mM is not captured. Leads to wrong updated attributes in containers")
            else:
                pass

        if log:
            self.container_logger.info(
                f"Container: dispensed {volume} uL into this container from source {source.WELL_ID} containing {source.concentration} mM {source.solution_name}."
            )